In [40]:
import urllib3
from bs4 import BeautifulSoup
import re  
import requests
import json


def getOrderedLinks(wikipage):
    http = urllib3.PoolManager()
    response = http.request("GET", wikipage)
    soup = BeautifulSoup(response.data)
    soup = soup.find(id="mw-content-text")#
    link_list = soup.find_all("a",href=re.compile("/wiki/"))
    return link_list

def getIncomingLinks(wikipage):
    S = requests.Session()
    URL = "https://de.wikipedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": wikipage,
        "bllimit": 500,
        "pldir" : "descending"
    }
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    BACKLINKS = DATA["query"]["backlinks"]
    return [link["title"] for link in BACKLINKS]
    
def cutIncomingLinks(incoming_links, base_link):
    output = []
    for link in incoming_links:
        all_links = getOrderedLinks(link)
        if(all_links.index(base_link) < 30):
            output.append(link)
    return output

def filterLinks(soup):
    reg = re.compile("^(?!/wiki/Datei:).+")
    reg2 = re.compile("^(?!/wiki/Spezial:).+")
    reg3 = re.compile("^/wiki")
    filteredList = [e for e in soup if reg.match(e["href"]) and reg2.match(e["href"]) and reg3.match(e["href"])]
    return filteredList

def cutList(linkList, length):
    return linkList[:length]

def requestClickData(linkList):
    out = dict()
    for i in linkList:
        out[i["title"]] = wikiPageView(i["title"])
    return out    

def wikiPageView(name):
    address = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/" + name + "/daily/20230701/20231001"
    # Personal username for identification for the Wikipedia API
    headers = {'User-Agent': 'marek.seipel@web.de'}
    resp = requests.get(address, headers=headers)
    details = resp.json()
    return details

def saveLinks(base_link, incoming_links, outgoing_links):
    data = {
        "baselink": base_link,
        "incoming_links": incoming_links,
        "outgoing_links": outgoing_links
    }
    with open("links_" + base_link +".json", "w") as link_file:
        link_file.write(json.dumps(data))
    
def filterTest(tagList):
    return [e for e in tagList if not "Wikipedia:" in e and not "Benutzer:" in e and not "Spezial:" in e and not "Diskussion:" in e and not "Portal:" in e]
    
#format for each base article: one json file with article as name
# incoming:{link:{daily views for whole year},link2...}
# outgoing:{link:{daily views for whole year},link2...}
def savePageview(pageview_list):
    print("not implemented yet")

In [11]:
import json 
with open('convert.txt', 'w') as convert_file: 
     convert_file.write(json.dumps(allData))


In [32]:
links = getIncomingLinks("Große_Anakonda")

In [42]:
filtered = filterTest(links)

def print_strings_with_colon(arr):
    for string in arr:
        if ":" in string:
            print(string)

print_strings_with_colon(filtered)

Anacondas: Die Jagd nach der Blut-Orchidee


In [22]:
reducedData = dict()
for key, value in allData.items():
    reducedData[key] = []
    for item in value["items"]:
        for key2, value2 in item.items():
            if key2 == "views":
                reducedData[key].append(value2)

In [23]:
reducedData

{'Staat': [262,
  307,
  313,
  335,
  330,
  243,
  282,
  288,
  283,
  337,
  360,
  319,
  255,
  319,
  246,
  298,
  316,
  270,
  300,
  285,
  265,
  252,
  267,
  314,
  298,
  274,
  286,
  265,
  304,
  317,
  285,
  281,
  275,
  275,
  273,
  265,
  291,
  339,
  274,
  283,
  313,
  295,
  276,
  250,
  332,
  428,
  411,
  353,
  265,
  232,
  295,
  371,
  310,
  324,
  355,
  291,
  240,
  289,
  329,
  340,
  357,
  327,
  296,
  231,
  298,
  286,
  426,
  360,
  336,
  322,
  270,
  270,
  387,
  340,
  383,
  356,
  306,
  292,
  296,
  418,
  381,
  364,
  343,
  338,
  305,
  275,
  383,
  378,
  335,
  331,
  336,
  246,
  275],
 'Staat Palästina': [680,
  675,
  1664,
  2059,
  1369,
  930,
  833,
  658,
  838,
  775,
  788,
  836,
  719,
  624,
  591,
  849,
  980,
  830,
  747,
  632,
  648,
  705,
  724,
  827,
  870,
  747,
  696,
  660,
  679,
  775,
  746,
  787,
  691,
  661,
  645,
  632,
  791,
  832,
  626,
  630,
  586,
  547,
  503,
  699,
  640,
  

In [44]:
views = wikiPageView("Große_Anakonda")

In [47]:
[{"date": e["timestamp"], "views": e["views"]} for e in views["items"]]

[{'project': 'de.wikipedia',
  'article': 'Große_Anakonda',
  'granularity': 'daily',
  'timestamp': '2023070100',
  'access': 'all-access',
  'agent': 'user',
  'views': 292},
 {'project': 'de.wikipedia',
  'article': 'Große_Anakonda',
  'granularity': 'daily',
  'timestamp': '2023070200',
  'access': 'all-access',
  'agent': 'user',
  'views': 288},
 {'project': 'de.wikipedia',
  'article': 'Große_Anakonda',
  'granularity': 'daily',
  'timestamp': '2023070300',
  'access': 'all-access',
  'agent': 'user',
  'views': 278},
 {'project': 'de.wikipedia',
  'article': 'Große_Anakonda',
  'granularity': 'daily',
  'timestamp': '2023070400',
  'access': 'all-access',
  'agent': 'user',
  'views': 362},
 {'project': 'de.wikipedia',
  'article': 'Große_Anakonda',
  'granularity': 'daily',
  'timestamp': '2023070500',
  'access': 'all-access',
  'agent': 'user',
  'views': 353},
 {'project': 'de.wikipedia',
  'article': 'Große_Anakonda',
  'granularity': 'daily',
  'timestamp': '2023070600',
